In [1]:
import pandas as pd

df = pd.read_csv('../dataset/op_int_equal_dataframe/op_int_equal_new.csv')
df.head()

,address,label,op_seq_0,op_seq_1,op_seq_2,op_seq_3,op_seq_4,op_seq_5,op_seq_6,op_seq_7,...,op_seq_16607,op_seq_16608,op_seq_16609,op_seq_16610,op_seq_16611,op_seq_16612,op_seq_16613,op_seq_16614,op_seq_16615,op_seq_16616
0,0x7fcc7ed28c99f64f721be410ad816247925aade8,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
1,0x60acd43314d93f38507f3fd12e84225a5a099229,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
2,0xbe181da06d93867af0a3ffadcebe345ec35dae4e,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
3,0x5e84c1a6e8b7cd42041004de5cd911d537c5c007,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
4,0x2464d1d97f8d0180cfad67bdb19bc30cca69dda0,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df.shape

(6718, 16619)

In [3]:
df.label.value_counts()

1    6535
0     183
Name: label, dtype: int64

In [4]:
type(df.iloc[0, :])

pandas.core.series.Series

In [6]:
# df.iloc[0, 760:len(df.iloc[0,2:])+2]

In [5]:
def get_first_0_index(df_row_series):
    index = 2
    for _, value in df_row_series.iteritems():
        if value == 0:
            return index
        index += 1
    return len(df_row_series)+2

In [7]:
# df.iloc[0, 2:get_first_0_index(df.iloc[0,2:])]
# raw_data = df.iloc[0, :]
# get_first_0_index(raw_data[2:])

In [8]:
# test_data = raw_data[2:772]
# test_data.shape[-1]

In [9]:
# slide contracts into slides
# dataset['address'] = [[slide_0], [slide_1], [slide_2], ..., [slide_n]]
import numpy as np

def sliding_window_split(raw_data, overlap=0, split_width=500):
    '''
    raw_data is the df_row Series with address and label
    return address, slices in np array, label
    '''
    
    # cut 0's
    zero_index = get_first_0_index(raw_data[2:])
    data = raw_data[2:zero_index]
    sample_width = data.shape[-1]
    index = 0
    splits = []

    while index + split_width < sample_width:
        splits.append(data[index:index + split_width])
        index += split_width - overlap

    if sample_width - index > split_width / 8:
        slice_from = -split_width if sample_width > split_width else 0
        splits.append(data[slice_from:])

    return raw_data[0], np.array(splits), raw_data[1]

In [10]:
# addr, test_slices, label = sliding_window_split(df.iloc[0,:])
# print(test_slices.shape)
# print(test_slices[0][0].dtype)
# print(type(test_slices))
# print(addr)
# print(label)


In [11]:
# [0] * test_slices.shape[0]

outside the class, split original df to train_df and test_df  
in the __init__, do the following things: 
1. build the giant dataset 
2. build ids or labels array
3. slice each contract and store in the giant dataset

In [12]:
def pad_zeros(np_array, build_width=500):
    '''for slice with smaller size, pad zeros and make them have standard size'''
    result = np.zeros(shape=(1,build_width), dtype=int)
    result[:np_array.shape[0],:np_array.shape[1]] = np_array
    return result

In [13]:
def build_dataset(dataframe, build_width=500):
    '''
        param: original datafeame
        return:
    '''
    count = 0
    all_data = np.zeros(shape=(1,build_width), dtype=int) # containing all slices
    labels = [] 
    print('start building dataset')
    for index, row in dataframe.iterrows():
        print('processing ', (count/len(dataframe))*100, '%, index ', count, ' / ', len(dataframe))
        count += 1
        curr_addr, cur_slices, cur_label = sliding_window_split(row)
        
#         print('cur slices shape: ', cur_slices.shape)
#         print('all data shape: ', all_data.shape)
        
#         print(type(cur_slices.shape))
#         print(type(cur_slices.shape[1]))
#         print(type(cur_slices.shape[1]))
#         print('cur slices content: ', cur_slices)
        
        if cur_slices.shape[0] > 0:
            if cur_slices.shape[1] < 500:
#                 print('old cur slices: ', cur_slices)
                cur_slices = pad_zeros(cur_slices, build_width)
#                 print('new cur slices: ', cur_slices)
#                 print('new cur slices shape: ', cur_slices.shape)

            all_data = np.concatenate((all_data, cur_slices), axis=0)
            for i in range(cur_slices.shape[0]):
                labels.append(cur_label)
        else:
#             print('zero found: ', cur_slices.shape)
#             print('zero content: ', cur_slices)
            print('empty contract found: ', curr_addr)
        

        
    all_data = all_data[1:]
    return all_data, labels

In [14]:
# all_data, labels = build_dataset(df)

# print(all_data.shape)
# print(len(labels))
# labels

In [15]:
# all_data[1].values

In [16]:
# 0~62 ponzi test dataset
# 63~182 ponzi train dataset
# 183~4553 non ponzi train dataset
# 4554~-1 non ponzi test dataset

train_df = df.iloc[63:4553, :]
ponzi_test = df.iloc[0:63, :]
non_ponzi_test = df.iloc[4553:, :]

# get the test and train df
test_df = pd.concat([ponzi_test, non_ponzi_test])
print(test_df.shape)
train_df

(2228, 16619)


,address,label,op_seq_0,op_seq_1,op_seq_2,op_seq_3,op_seq_4,op_seq_5,op_seq_6,op_seq_7,...,op_seq_16607,op_seq_16608,op_seq_16609,op_seq_16610,op_seq_16611,op_seq_16612,op_seq_16613,op_seq_16614,op_seq_16615,op_seq_16616
63,0xcff9cb72d19c10df754ae7be6d280e379cdb2354,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
64,0xa7481af3bbc9d902c02f58204c5683052de53330,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
65,0x4fb663c1616bfe80b5b6d5a214efa81d5a121801,0,61,51,34,21,62,56,64,61,...,0,0,0,0,0,0,0,0,0,0
66,0xf77ac34cd0ed42f2b5d8cfcf4fa0f4bb1b80b9d8,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
67,0xb5aff01f3e820735fac9abb594dc9993cb9e5bd2,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
68,0x40b88bbd78cb0441259de7a7d608a39b7388369a,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
69,0x85c84c949b1c29c85b777f2873a19450dbb1a057,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
70,0x9f1d916a456b96146e9f0dbbd0e107a1f389a061,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
71,0xadd6f226b34957a8f59a5be195560b96323c74c7,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0
72,0xb1d58bad78f33892719cdeba218f8641a71a3f05,0,61,51,34,21,62,56,61,61,...,0,0,0,0,0,0,0,0,0,0


In [17]:
import torch
from torch import nn
from torch.utils import data as torch_data
import torch.nn.functional as F

class PandasReader(torch_data.Dataset):

    def __init__(self, dataset, labels):
        self.data = dataset
        self.target = labels
        self.shape = self.data.shape

    def __getitem__(self, i):
        features = torch.from_numpy(self.data[i].astype(int))
        target = torch.tensor(self.target[i])
        return features, target

    def __len__(self):
        return self.data.shape[0]

In [32]:
class RNN(nn.Module):

    def __init__(self, input_dim, unique_codes):
        super(RNN, self).__init__()
        self. emb = torch.nn.Embedding(unique_codes, 64)
        self.lstm_1 = nn.LSTM(input_size=64, hidden_size=64, num_layers=2, batch_first=True)
        self.fc_1 = nn.Linear(in_features=64, out_features=16)
        self.fc_2 = nn.Linear(in_features=16, out_features=1)
        self.dropout = nn.Dropout()
        self.hidden = None


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = self.emb(x)
        y, self.hidden = self.lstm_1(x, self.hidden)
        y = self.dropout(y[:,-1])
        y = self.dropout(F.leaky_relu(self.fc_1(y)))
        y = self.dropout(F.leaky_relu(self.fc_2(y)))
        return y

In [30]:
def train(train_loader, val_loader, epochs):
    model = RNN(train_loader.dataset.shape[1], 262).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(epochs):
        train_loss = 0.0
        val_loss = 0.0
        
        count = 0
        
        for _, (features, target) in enumerate(train_loader):
            
            print(f'processing {count/len(train_loader)}, {count}/{len(train_loader)}')
            count += 1
            
            features = features.to(device)
            #import pdb; pdb.set_trace()
            target = target.to(device).double()
            
            print(40*'-')
            print('Target')
            print(target)
            print(40*'-')

            x = features
            y = model.forward(x).squeeze(dim=-1)
            loss = criterion(y, target)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            train_loss += loss.item()
            optimizer.step()
            print(f"EPOCH {epoch+1}: TRAIN LOSS - {train_loss}")
        train_loss = train_loss / len(train_loader)

        with torch.no_grad():
            model.eval()
            for _, (features, target) in enumerate(val_loader):
                x = features.unsqueeze(dim=0)
                preds = model.forward(x).squeeze(dim=0)
                loss = criterion(preds, target)
                val_loss += loss.item()
            model.train()
            val_loss = val_loss / len(val_loader)

        print(f"EPOCH {epoch+1}: TRAIN LOSS - {train_loss}, VAL LOSS - {val_loss}")

    return model

In [21]:
# train_loader.dataset[0]
# train_loader.dataset[591][1]

In [22]:
# train_loader.dataset.target

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("USING DEVICE:", device)

#     torch.set_default_tensor_type("torch.DoubleTensor")
torch.set_default_dtype(torch.float64)

train_dataset, train_labels = build_dataset(train_df)
test_dataset, test_labels = build_dataset(test_df)
#     train_reader = PandasReader()

#     df_train = pd.read_csv("./train.csv").drop("Date", axis="columns")
#     train_loader = torch_data.DataLoader(PandasReader(train_df), batch_size=30, shuffle=True)

#     df_val = pd.read_csv("./val.csv").drop("Date", axis="columns")
#     val_loader = torch_data.DataLoader(PandasReader(test_df), batch_size=30, shuffle=False)

#     print("TRAINING DATA SHAPE:", train_loader.dataset.shape)
#     print("VALIDATION DATA SHAPE:", val_loader.dataset.shape)

#     model = train(train_loader, val_loader, epochs=50)


USING DEVICE: cpu
start building dataset
processing  1.4031180400890868 , index  63  /  4490
processing  1.4253897550111359 , index  64  /  4490
processing  1.447661469933185 , index  65  /  4490
processing  1.469933184855234 , index  66  /  4490
processing  1.4922048997772828 , index  67  /  4490
processing  1.5144766146993318 , index  68  /  4490
processing  1.5367483296213809 , index  69  /  4490
processing  1.55902004454343 , index  70  /  4490
processing  1.581291759465479 , index  71  /  4490
processing  1.6035634743875278 , index  72  /  4490
processing  1.6258351893095768 , index  73  /  4490
processing  1.6481069042316259 , index  74  /  4490
processing  1.670378619153675 , index  75  /  4490
processing  1.6926503340757237 , index  76  /  4490
processing  1.7149220489977728 , index  77  /  4490
processing  1.7371937639198218 , index  78  /  4490
processing  1.7594654788418707 , index  79  /  4490
processing  1.7817371937639197 , index  80  /  4490
processing  1.804008908685968

processing  4.922048997772829 , index  221  /  4490
processing  4.944320712694878 , index  222  /  4490
processing  4.966592427616927 , index  223  /  4490
processing  4.988864142538976 , index  224  /  4490
processing  5.011135857461024 , index  225  /  4490
processing  5.033407572383074 , index  226  /  4490
processing  5.055679287305122 , index  227  /  4490
processing  5.077951002227171 , index  228  /  4490
processing  5.1002227171492205 , index  229  /  4490
processing  5.122494432071269 , index  230  /  4490
processing  5.144766146993319 , index  231  /  4490
processing  5.167037861915367 , index  232  /  4490
processing  5.189309576837417 , index  233  /  4490
processing  5.211581291759466 , index  234  /  4490
processing  5.233853006681515 , index  235  /  4490
processing  5.256124721603563 , index  236  /  4490
processing  5.278396436525613 , index  237  /  4490
processing  5.3006681514476615 , index  238  /  4490
processing  5.32293986636971 , index  239  /  4490
processing 

processing  8.552338530066814 , index  384  /  4490
processing  8.574610244988865 , index  385  /  4490
processing  8.596881959910913 , index  386  /  4490
processing  8.619153674832962 , index  387  /  4490
processing  8.641425389755012 , index  388  /  4490
processing  8.663697104677059 , index  389  /  4490
processing  8.68596881959911 , index  390  /  4490
processing  8.708240534521158 , index  391  /  4490
processing  8.730512249443208 , index  392  /  4490
processing  8.752783964365255 , index  393  /  4490
processing  8.775055679287306 , index  394  /  4490
processing  8.797327394209354 , index  395  /  4490
processing  8.819599109131403 , index  396  /  4490
processing  8.841870824053451 , index  397  /  4490
processing  8.864142538975502 , index  398  /  4490
processing  8.88641425389755 , index  399  /  4490
processing  8.908685968819599 , index  400  /  4490
processing  8.93095768374165 , index  401  /  4490
processing  8.953229398663696 , index  402  /  4490
processing  8.9

processing  12.138084632516703 , index  545  /  4490
processing  12.160356347438753 , index  546  /  4490
processing  12.182628062360802 , index  547  /  4490
processing  12.20489977728285 , index  548  /  4490
processing  12.2271714922049 , index  549  /  4490
processing  12.24944320712695 , index  550  /  4490
processing  12.271714922048996 , index  551  /  4490
processing  12.293986636971047 , index  552  /  4490
processing  12.316258351893095 , index  553  /  4490
processing  12.338530066815146 , index  554  /  4490
processing  12.360801781737193 , index  555  /  4490
processing  12.383073496659243 , index  556  /  4490
processing  12.405345211581292 , index  557  /  4490
processing  12.42761692650334 , index  558  /  4490
processing  12.44988864142539 , index  559  /  4490
processing  12.47216035634744 , index  560  /  4490
processing  12.494432071269488 , index  561  /  4490
processing  12.516703786191538 , index  562  /  4490
processing  12.538975501113587 , index  563  /  4490


processing  15.701559020044542 , index  705  /  4490
processing  15.723830734966594 , index  706  /  4490
processing  15.74610244988864 , index  707  /  4490
processing  15.768374164810691 , index  708  /  4490
processing  15.79064587973274 , index  709  /  4490
processing  15.812917594654788 , index  710  /  4490
processing  15.835189309576839 , index  711  /  4490
processing  15.857461024498887 , index  712  /  4490
processing  15.879732739420934 , index  713  /  4490
processing  15.902004454342986 , index  714  /  4490
processing  15.924276169265033 , index  715  /  4490
processing  15.946547884187082 , index  716  /  4490
processing  15.968819599109132 , index  717  /  4490
processing  15.99109131403118 , index  718  /  4490
processing  16.01336302895323 , index  719  /  4490
processing  16.035634743875278 , index  720  /  4490
processing  16.057906458797326 , index  721  /  4490
processing  16.080178173719375 , index  722  /  4490
processing  16.102449888641427 , index  723  /  44

...

processing  267.9084380610413 , index  5969  /  2228
processing  267.95332136445245 , index  5970  /  2228
processing  267.99820466786355 , index  5971  /  2228
processing  268.0430879712747 , index  5972  /  2228
processing  268.0879712746858 , index  5973  /  2228
processing  268.13285457809695 , index  5974  /  2228
processing  268.1777378815081 , index  5975  /  2228
processing  268.2226211849192 , index  5976  /  2228
processing  268.26750448833036 , index  5977  /  2228
processing  268.3123877917415 , index  5978  /  2228
processing  268.3572710951526 , index  5979  /  2228
processing  268.40215439856377 , index  5980  /  2228
processing  268.44703770197486 , index  5981  /  2228
processing  268.491921005386 , index  5982  /  2228
processing  268.5368043087971 , index  5983  /  2228
processing  268.58168761220827 , index  5984  /  2228
processing  268.62657091561937 , index  5985  /  2228
processing  268.6714542190305 , index  5986  /  2228
processing  268.7163375224416 , index  

processing  274.82046678635544 , index  6123  /  2228
processing  274.8653500897666 , index  6124  /  2228
processing  274.91023339317775 , index  6125  /  2228
processing  274.95511669658885 , index  6126  /  2228
processing  275.0 , index  6127  /  2228
processing  275.04488330341115 , index  6128  /  2228
processing  275.08976660682225 , index  6129  /  2228
processing  275.1346499102334 , index  6130  /  2228
processing  275.17953321364456 , index  6131  /  2228
processing  275.22441651705566 , index  6132  /  2228
processing  275.2692998204668 , index  6133  /  2228
processing  275.3141831238779 , index  6134  /  2228
processing  275.35906642728906 , index  6135  /  2228
processing  275.4039497307002 , index  6136  /  2228
processing  275.4488330341113 , index  6137  /  2228
processing  275.4937163375224 , index  6138  /  2228
processing  275.53859964093357 , index  6139  /  2228
processing  275.58348294434467 , index  6140  /  2228
processing  275.6283662477558 , index  6141  /  

processing  281.8222621184919 , index  6279  /  2228
processing  281.86714542190305 , index  6280  /  2228
processing  281.9120287253142 , index  6281  /  2228
processing  281.9569120287253 , index  6282  /  2228
processing  282.00179533213645 , index  6283  /  2228
processing  282.04667863554755 , index  6284  /  2228
processing  282.0915619389587 , index  6285  /  2228
processing  282.13644524236986 , index  6286  /  2228
processing  282.18132854578096 , index  6287  /  2228
processing  282.2262118491921 , index  6288  /  2228
processing  282.27109515260327 , index  6289  /  2228
processing  282.31597845601436 , index  6290  /  2228
processing  282.3608617594255 , index  6291  /  2228
processing  282.4057450628367 , index  6292  /  2228
processing  282.45062836624777 , index  6293  /  2228
processing  282.49551166965887 , index  6294  /  2228
processing  282.54039497307 , index  6295  /  2228
processing  282.5852782764811 , index  6296  /  2228
processing  282.6301615798923 , index  

processing  288.77917414721725 , index  6434  /  2228
processing  288.82405745062835 , index  6435  /  2228
processing  288.8689407540395 , index  6436  /  2228
processing  288.9138240574506 , index  6437  /  2228
processing  288.95870736086175 , index  6438  /  2228
processing  289.0035906642729 , index  6439  /  2228
processing  289.048473967684 , index  6440  /  2228
processing  289.09335727109516 , index  6441  /  2228
processing  289.1382405745063 , index  6442  /  2228
processing  289.1831238779174 , index  6443  /  2228
processing  289.22800718132856 , index  6444  /  2228
processing  289.2728904847397 , index  6445  /  2228
processing  289.3177737881508 , index  6446  /  2228
processing  289.36265709156197 , index  6447  /  2228
processing  289.40754039497307 , index  6448  /  2228
processing  289.4524236983842 , index  6449  /  2228
processing  289.4973070017953 , index  6450  /  2228
processing  289.5421903052065 , index  6451  /  2228
processing  289.5870736086176 , index  6

processing  295.78096947935364 , index  6590  /  2228
processing  295.8258527827648 , index  6591  /  2228
processing  295.87073608617595 , index  6592  /  2228
processing  295.91561938958705 , index  6593  /  2228
processing  295.9605026929982 , index  6594  /  2228
processing  296.00538599640936 , index  6595  /  2228
processing  296.05026929982046 , index  6596  /  2228
processing  296.0951526032316 , index  6597  /  2228
processing  296.1400359066427 , index  6598  /  2228
processing  296.18491921005386 , index  6599  /  2228
processing  296.229802513465 , index  6600  /  2228
processing  296.2746858168761 , index  6601  /  2228
processing  296.31956912028727 , index  6602  /  2228
processing  296.3644524236984 , index  6603  /  2228
processing  296.4093357271095 , index  6604  /  2228
processing  296.4542190305207 , index  6605  /  2228
processing  296.4991023339318 , index  6606  /  2228
processing  296.54398563734287 , index  6607  /  2228
processing  296.588868940754 , index  6

In [24]:
train_reader = PandasReader(train_dataset, train_labels)
test_reader = PandasReader(test_dataset, test_labels)

train_loader = torch_data.DataLoader(train_reader, batch_size=30, shuffle=True, drop_last=True)
val_loader = torch_data.DataLoader(test_reader, batch_size=30, shuffle=False, drop_last=True)

print("TRAINING DATA SHAPE:", train_loader.dataset.shape)
print("VALIDATION DATA SHAPE:", val_loader.dataset.shape)

TRAINING DATA SHAPE: (29953, 500)
VALIDATION DATA SHAPE: (14605, 500)


In [25]:
train_loader.dataset[0]

(tensor([ 61,  51,  34,  21,  62,  56,  61,  61,  11,  61,  33,   5,  64,  94,
          20,  62,  56,  93,  64,  20,  62,  56,  93,  64,  20,  62,  56,  93,
          64,  20,  62,  56,  93,  64,  20,  62,  56,  93,  64,  20,  62,  56,
          93,  64,  20,  62,  56,  60,  62,  62,  62,  55,  60,  62,  61,  33,
          61,  53,  61,  61,  61,  11,   4, 109,  94,  22,  31, 110, 109, 110,
          22,  20,  21,  62,  56,  61,  93,  53,  61,  61,  61,  11,   4,  25,
          22,  95,  23, 109,  54,  60,  49,  55,  60,  62,  61,  53,  61,  61,
          61,  11,   4,  22,  94,  55,  60,  62,  61,  53,  94,  55,  60,  62,
          61,  33,  61,  93,  53,  95, 109,  94,  16,  21,  62,  56,  49,  93,
          51,  61,   3,  92,  94,   2,  53,  61,  93,  50,  61,  62,  96,  37,
          94,  50, 110,  51, 110, 109, 110,   2,  53,  61,  61,  61,  11,   4,
         110, 109, 110,  22, 109,  95,  55,  60,  62,  61,  53,  94,  55,  60,
          62,  61,  53,  94,  55,  60,  62,  60,  61

In [26]:
train_loader.dataset.target

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
model = train(train_loader, val_loader, epochs=50)
